# Basic settings

In [ ]:
# name of the directory containing weights
name = "pseudo_weights"
# name of the individual weights
weights = ["fold0", "fold1", "fold2", "fold3", "new_fold3"]
# names of the branches used to inference individual weights
branches = ["original", "original", "original", "original", "master"]
# indexes of labels whose confidences have to be *sqrt*ed
sqrt_indexes = [4]

# Get individual labels

In [ ]:
for index, weight in enumerate(weights):
    print(index)
    !cd GWC_YOLOv5 && git checkout {branches[index]} 
    !cd GWC_YOLOv5 && python detect.py --img-size 800 --name {weight} --weights ../{name}/{weight}.pt --source ../test --augment --nosave --save-txt --save-conf --conf-thres 0.50
    !cp -r GWC_YOLOv5/runs/detect/{weight}/labels {weight}

# Weighted boxes fusion
Use [weighted boxes fusion](https://github.com/ZFTurbo/Weighted-Boxes-Fusion) to ensemble individual labels for further training.

In [ ]:
import os

files = [os.listdir(folder) for folder in weights]

In [ ]:
import numpy as np


def parse_box(box):
    x, y, w, h = box
    return max(0, x - w / 2), max(0, y - h / 2), min(1, x + w / 2), min(1, y + h / 2)


def parse_file(file_name, current):
    boxes = []
    scores = []
    labels = []
    with open(file_name) as f:
        for line in f:
            cls, *xywh, conf = list(map(float, line.split()))
            boxes.append(parse_box(xywh))
            if current in sqrt_indexes:
                scores.append(np.sqrt(conf))
            else:
                scores.append(conf)
            labels.append(0)
    return boxes, scores, labels

In [ ]:
import os
import shutil

from ensemble_boxes import weighted_boxes_fusion
from tqdm import tqdm

if os.path.exists("labels"):
    shutil.rmtree("labels")
os.mkdir("labels")

for img_name in tqdm(os.listdir("test")):
    txt_name = img_name[:-4] + ".txt"
    boxes_list = [[] for _ in range(len(folders))]
    scores_list = [[] for _ in range(len(folders))]
    labels_list = [[] for _ in range(len(folders))]
    weights_list = [1 for _ in range(len(folders))]
    for current, folder in enumerate(folders):
        if txt_name in files[current]:
            boxes, scores, labels = parse_file(os.path.join(folder, txt_name), current)
            boxes_list[current] = boxes
            scores_list[current] = scores
            labels_list[current] = labels
    final_boxes, final_scores, _ = weighted_boxes_fusion(
        boxes_list, scores_list, labels_list, weights=weights_list
    )

    with open("labels/" + txt_name, "w") as f:
        for box in final_boxes:
            x1, y1, x2, y2 = box
            x_center = (x1 + x2) / 2
            y_center = (y1 + y2) / 2
            width = x2 - x1
            height = y2 - y1
            temp = " ".join(
                ["0", str(x_center), str(y_center), str(width), str(height)]
            )
            f.write(temp)
            f.write("\n")

assert len(os.listdir("test")) == len(os.listdir("labels"))